In [1]:
# This is part-1 of my back-testing code to test the momentum stock picking process
# The code will pick top-10 stocks from last 4 years based on momentum strength and momentum quality

import numpy as np
import yfinance as yf
import pandas as pd

def get_sp500_tickers():
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    tickers = table[0]['Symbol'].tolist()
    return tickers

def fetch_and_analyze_data(ticker, start_date, end_date):
    try:
        adjusted_ticker = ticker.replace('.', '-')
        stock_data = yf.Ticker(adjusted_ticker).history(start=start_date, end=end_date)

        if not stock_data.empty:
            stock_data.index = stock_data.index.tz_localize(None)
            
            start_price = stock_data.iloc[0]['Close']
            end_price = stock_data.iloc[-1]['Close']
            annual_return = (end_price - start_price) / start_price * 100
            
            stock_data['Daily Return'] = stock_data['Close'].pct_change()
            std_dev = stock_data['Daily Return'].std() * (252**0.5)
            sharpe_ratio = (annual_return / 100 - risk_free_rate) / std_dev
            
            # Fetch P/E ratio using yfinance
            pe_ratio = yf.Ticker(adjusted_ticker).info.get('trailingPE', None)

            # Calculate Momentum Health Indicator
            stock_data['up_days'] = stock_data['Daily Return'] > 0
            stock_data['down_days'] = stock_data['Daily Return'] < 0

            up_days_percentage = stock_data['up_days'].sum() / len(stock_data) * 100
            down_days_percentage = stock_data['down_days'].sum() / len(stock_data) * 100

            stock_data['Momentum Health Indicator'] = up_days_percentage - down_days_percentage

            # Drop the temporary columns
            stock_data.drop(columns=['up_days', 'down_days'], inplace=True)
            
            # Calculate Delta-Normal VAR at 95% confidence level
            expected_return = stock_data['Daily Return'].mean()
            var_95 = expected_return - 1.96 * std_dev
            stock_data['VAR'] = var_95

            return {'Ticker': ticker, 'Return': annual_return, 'Std Dev': std_dev, 'Sharpe Ratio': sharpe_ratio, 'P/E Ratio': pe_ratio, 'Momentum Health Indicator': stock_data['Momentum Health Indicator'].iloc[-1], 'VAR': stock_data['VAR'].iloc[-1]}, stock_data
        else:
            return None, None
    except Exception as e:
        print(f"Skipping {ticker} due to error: {e}")
        return None, None

def process_year(year, start_date, end_date):
    returns_data = []
    for ticker in sp500_tickers:
        result, stock_data = fetch_and_analyze_data(ticker, start_date, end_date)
        if result and stock_data is not None:
            returns_data.append(result)
            stock_data.to_excel(writer, sheet_name=f'{ticker}_{year}')
    returns_df = pd.DataFrame(returns_data)

    # Convert P/E Ratio to numeric, forcing errors to NaN
    returns_df['P/E Ratio'] = pd.to_numeric(returns_df['P/E Ratio'], errors='coerce')

    # Drop rows with NaN P/E Ratio
    returns_df = returns_df.dropna(subset=['P/E Ratio'])

    # Filter out extremely high P/E Ratios that might skew the average
    returns_df = returns_df[returns_df['P/E Ratio'] < 1000]

    average_pe_ratio = returns_df['P/E Ratio'].mean()
    print(f"Average PE Ratio for {year}: ", average_pe_ratio, "\n")

    filtered_df = returns_df[(returns_df['P/E Ratio'] < average_pe_ratio) & (returns_df['Momentum Health Indicator'] > 10)]
    filtered_df_sorted = filtered_df.sort_values(by='Return', ascending=False)
    top_10_stocks = filtered_df_sorted.head(10)
    top_10_stocks.to_excel(writer, sheet_name=f'Top 10 Stocks {year}', index=False)
    return top_10_stocks

sp500_tickers = get_sp500_tickers()
risk_free_rate = 0.02
writer = pd.ExcelWriter('sp500_data.xlsx', engine='openpyxl')

years = {
    '2021': ('2020-12-31', '2021-12-31'),
    '2022': ('2021-12-31', '2022-12-31'),
    '2023': ('2022-12-31', '2023-12-31'),
    '2024': ('2023-12-29', '2024-12-31')
}

for year, (start_date, end_date) in years.items():
    print(f"Processing data for {year}...")
    top_10_stocks = process_year(year, start_date, end_date)
    print(top_10_stocks[['Ticker', 'Return', 'Std Dev', 'Sharpe Ratio', 'P/E Ratio', 'Momentum Health Indicator', 'VAR']])

writer.close()

Processing data for 2021...


$CEG: possibly delisted; no price data found  (1d 2020-12-31 -> 2021-12-31) (Yahoo error = "Data doesn't exist for startDate = 1609390800, endDate = 1640926800")
$GEHC: possibly delisted; no price data found  (1d 2020-12-31 -> 2021-12-31) (Yahoo error = "Data doesn't exist for startDate = 1609390800, endDate = 1640926800")
$GEV: possibly delisted; no price data found  (1d 2020-12-31 -> 2021-12-31) (Yahoo error = "Data doesn't exist for startDate = 1609390800, endDate = 1640926800")
$KVUE: possibly delisted; no price data found  (1d 2020-12-31 -> 2021-12-31) (Yahoo error = "Data doesn't exist for startDate = 1609390800, endDate = 1640926800")
$SW: possibly delisted; no price data found  (1d 2020-12-31 -> 2021-12-31) (Yahoo error = "Data doesn't exist for startDate = 1609390800, endDate = 1640926800")
$SOLV: possibly delisted; no price data found  (1d 2020-12-31 -> 2021-12-31) (Yahoo error = "Data doesn't exist for startDate = 1609390800, endDate = 1640926800")
$VLTO: possibly delisted; 

Average PE Ratio for 2021:  35.74748107723404 

    Ticker      Return   Std Dev  Sharpe Ratio  P/E Ratio  \
343    NUE  117.872329  0.417050      2.778381  11.635577   
76    BLDR  105.954423  0.378237      2.748391  14.831213   
406    STX   88.830699  0.390729      2.222275  23.322998   
170   EPAM   87.844287  0.360966      2.378181  29.595455   
315    MAA   85.467342  0.189284      4.409624  33.820225   
40    AMAT   84.360461  0.439212      1.875185  20.105110   
349   ODFL   83.064368  0.241311      3.359331  31.745200   
82     CPT   82.668832  0.191470      4.213130  35.381702   
442   TSCO   75.770505  0.255957      2.882148  26.342234   
378    PLD   71.930905  0.192423      3.634219  32.154080   

     Momentum Health Indicator       VAR  
343                  13.095238 -0.813968  
76                   16.269841 -0.738180  
406                  16.269841 -0.762994  
170                  13.888889 -0.704727  
315                  19.444444 -0.368463  
40                   1

$GEV: possibly delisted; no price data found  (1d 2021-12-31 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1672462800")
$KVUE: possibly delisted; no price data found  (1d 2021-12-31 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1672462800")
$SW: possibly delisted; no price data found  (1d 2021-12-31 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1672462800")
$SOLV: possibly delisted; no price data found  (1d 2021-12-31 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1672462800")
$VLTO: possibly delisted; no price data found  (1d 2021-12-31 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1672462800")


Average PE Ratio for 2022:  35.67305658389831 

    Ticker     Return   Std Dev  Sharpe Ratio  P/E Ratio  \
186    XOM  87.409088  0.350870      2.434206  13.556788   
297    MPC  86.622796  0.362402      2.335050  11.371895   
462    VLO  74.953926  0.448954      1.624977  11.512314   
101    CVX  58.462348  0.329204      1.715119  17.036263   
309    MRK  49.421389  0.199304      2.379352  20.725940   
43     ADM  39.979772  0.311340      1.219879  14.435393   
146   FANG  35.337606  0.471191      0.707518  10.261973   
83     CPB  34.843026  0.238685      1.375996  21.158470   
488    WMB  32.831247  0.288110      1.070122  23.991526   
213    GIS  28.092334  0.224177      1.163917  12.865218   

     Momentum Health Indicator       VAR  
186                  15.476190 -0.684956  
297                  12.301587 -0.707560  
462                  10.714286 -0.877319  
101                  13.888889 -0.643189  
309                  11.904762 -0.388956  
43                   10.714286 -0

...

Skipping IFF due to error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


$PSX: possibly delisted; no price data found  (1d 2022-12-31 -> 2023-12-31)
$SW: possibly delisted; no price data found  (1d 2022-12-31 -> 2023-12-31) (Yahoo error = "Data doesn't exist for startDate = 1672462800, endDate = 1703998800")
$SOLV: possibly delisted; no price data found  (1d 2022-12-31 -> 2023-12-31) (Yahoo error = "Data doesn't exist for startDate = 1672462800, endDate = 1703998800")


Average PE Ratio for 2023:  35.760986647991544 

    Ticker      Return   Std Dev  Sharpe Ratio  P/E Ratio  \
423   SMCI  238.969731  0.751333      3.153990  15.689054   
402    RCL  165.838665  0.364839      4.490713  22.978960   
76    BLDR  155.455251  0.417931      3.671780  14.848825   
452   UBER  142.783905  0.361867      3.890491  32.595173   
384    PHM  125.444357  0.300122      4.113136   8.161623   
208     GE   92.971734  0.242983      3.743955  34.295277   
260    JBL   90.273934  0.365913      2.412433  14.640831   
286    LII   87.906619  0.291485      2.947208  29.799334   
68    BKNG   74.549864  0.249525      2.907515  32.469055   
155    DHI   69.149812  0.281647      2.384182   9.873431   

     Momentum Health Indicator       VAR  
423                       13.2 -1.466602  
402                       14.4 -0.710890  
76                        16.4 -0.815027  
452                       12.4 -0.705434  
384                       17.2 -0.584793  
208                  

$BAX: possibly delisted; no price data found  (1d 2023-12-29 -> 2024-12-31)


Average PE Ratio for 2024:  36.041180601260514 

    Ticker      Return   Std Dev  Sharpe Ratio  P/E Ratio  \
475    VST  267.350517  0.567212      4.678152  31.752820   
219   GDDY   87.876780  0.233776      3.673457  15.905698   
38     APO   81.345752  0.319786      2.481211  16.685850   
202    FOX   69.504967  0.204744      3.297037  11.408313   
201   FOXA   67.198062  0.210526      3.096911  11.964547   
207     GE   66.120068  0.305103      2.101587  34.411438   
492    WMB   62.469519  0.195985      3.085409  24.216103   
26     AXP   60.611815  0.241544      2.426549  21.981619   
448     TT   53.284578  0.233322      2.198015  35.247932   
65      BK   52.454041  0.177024      2.850129  17.029083   

     Momentum Health Indicator       VAR  
475                  18.650794 -1.105908  
219                  22.619048 -0.455579  
38                   13.888889 -0.624205  
202                  16.269841 -0.399112  
201                  13.095238 -0.410494  
207                  